In [1]:
#Necessary packages
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from math import sqrt
import datetime 
import calendar
import sys

In [2]:
# Load the data from the Excel file
file_path = r'C:\Users\PHIRI003\OneDrive - Wageningen University & Research\Documents\WEcR Internship\Work\Dairy\Merged_data.xlsx'
df = pd.read_excel(file_path, header=0 )

df.index = df['Date']

In [3]:
# Define the function to create seasonal dummy variables
def SeasonalDummies(df, frequency='M'):
    nT = len(df)  # Number of observations
    startdate = df.index[0]  # Start date of the time series
    datetime = pd.DataFrame(data=pd.date_range(startdate, periods=nT, freq=frequency), columns=["datetime"])
    monthnumber = datetime["datetime"].dt.month  # Extract month numbers
    monthname = pd.DataFrame()  # DataFrame to store month names
    
    for i in range(nT):
        monthname.at[i, 'D'] = calendar.month_name[monthnumber[i]]  # Assign month names based on month numbers
    
    seasdum = pd.get_dummies(monthname)  # Create dummy variables for each month
    seasdum = seasdum.drop('D_January', axis=1)  # Drop January to avoid multicollinearity
    seasdum = seasdum.set_index(df.index)  # Set the index to match the original DataFrame
    seasdum = seasdum.astype(int)
    return seasdum

# Create seasonal dummies
seasdum = SeasonalDummies(df)

# Combine the data with the seasonal dummy variables
df = pd.concat([df, seasdum], axis=1)


In [8]:
#Create a break dummy for a break point
break_date = '2015-01-01'
#break_date = '2010-11-01'
#break_date = '2022-12-01'

df['break_dummy'] = (df['Date']>= break_date).astype(int)

#create interaction terms between  break dummy and price indices
df['Inter_API'] = df['break_dummy']*df['API'] 
df['Inter_PPI'] = df['break_dummy']*df['PPI'] 
df['Inter_CPI'] = df['break_dummy']*df['CPI'] 

#create a trend variable
df['trend'] = range(1, len(df) + 1)  

Test for long-run relationships among the key variables API, PPI and CPI
Note: Gregory and Hansen test allows for more than one independent variables

In [ ]:
from statsmodels.tsa.stattools import adfuller

#Gregory and Handen cointegration test - CPI
# Create a list of independent variables
X = df[['trend','API', 'PPI', 'break_dummy', 'Inter_API', 'Inter_PPI', 'D_April', 'D_August', 'D_December', 
        'D_February',  'D_July',  'D_June',  'D_March', 'D_May', 'D_September', 'D_October', 'D_November']] 
X = sm.add_constant(X)
Y = df['CPI']

# long -run OLS estimation
ols_model = sm.OLS(Y,X).fit()
print(ols_model.summary())

#Cointegration test - Gregory and Hansen
df['residuals'] = ols_model.resid

# Perform the Augmented Dickey-Fuller test on the residuals
adf_test = adfuller(df['residuals'], maxlag=12)

# Print the results
print('ADF statistic:', adf_test[0])


In [ ]:
from statsmodels.tsa.stattools import adfuller

#Gregory and Handen cointegration test -PPI
X2 = df[['trend','API', 'CPI', 'break_dummy', 'Inter_API', 'Inter_CPI', 'D_April', 'D_August', 'D_December', 'D_February',  'D_July',  'D_June',  
         'D_March', 'D_May', 'D_September', 'D_October', 'D_November']] 
X2 = sm.add_constant(X2)
Y2 = df['PPI']

#OLS estimation
ols_model2 = sm.OLS(Y2,X2).fit()
print(ols_model2.summary())

#Cointegration test - Gregory and Hansen
# Assuming 'df_with_dummies' is already defined and 'ols_model' is fitted
df['residuals2'] = ols_model2.resid

# Perform the Augmented Dickey-Fuller test on the residuals
adf_test2 = adfuller(df['residuals2'], maxlag=12)

# Print the results
print('ADF statistic:', adf_test2[0])

In [ ]:
from statsmodels.tsa.stattools import adfuller

#Gregory and Handen cointegration test - API
X3 = df[['trend','PPI', 'CPI', 'break_dummy', 'Inter_API', 'Inter_CPI', 'D_April', 'D_August', 'D_December', 'D_February',  'D_July',  'D_June',  'D_March', 'D_May', 'D_September', 'D_October', 'D_November']] 
#X = df_with_dummies[['trend','API']] 
X3 = sm.add_constant(X3)
Y3 = df['API']

#OLS estimation
ols_model3 = sm.OLS(Y3,X3).fit()
print(ols_model3.summary())

#Cointegration test - Gregory and Hansen
# Assuming 'df_with_dummies' is already defined and 'ols_model' is fitted
df['residuals3'] = ols_model3.resid

# Perform the Augmented Dickey-Fuller test on the residuals
adf_test3 = adfuller(df['residuals3'], maxlag=12)

# Print the results
print('ADF statistic:', adf_test3[0])